In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta

DATA_PATH = "../data/"
SAVE_PATH = "../charts/"

START_DATE = datetime(2005, 7, 1)

In [ ]:
# Загружаем данные

beta = pd.read_excel(DATA_PATH + "beta_gibbs.xlsx", header=None).values
Xmat = pd.read_excel(DATA_PATH + "X.xlsx", header=None).values
shocks_AS = pd.read_excel(DATA_PATH + "ETA_record AS.xlsx", header=None).values
shocks_PrivateAD = pd.read_excel(DATA_PATH + "ETA_record PrivateAD.xlsx", header=None).values
shocks_MP = pd.read_excel(DATA_PATH + "ETA_record MP.xlsx", header=None).values
shocks_FP = pd.read_excel(DATA_PATH + "ETA_record FP.xlsx", header=None).values
A0inv_p = pd.read_excel(DATA_PATH + "struct_irf_record for p.xlsx", header=None).values
A0inv_y = pd.read_excel(DATA_PATH + "struct_irf_record for Y.xlsx", header=None).values
A0inv_i = pd.read_excel(DATA_PATH + "struct_irf_record for i.xlsx", header=None).values
A0inv_b = pd.read_excel(DATA_PATH + "struct_irf_record for B.xlsx", header=None).values

actual_p = pd.read_excel(DATA_PATH + 'actual_p.xlsx', header=None).values

In [ ]:
# Задаем параметры модели
T = Xmat.shape[0]
N_ITER = beta.shape[1]
N_VARS = 4
N_LAGS = 2
N_REG = 10

# Индексы переменных
P = 1 - 1
Y = 2 - 1
I = 3 - 1
B = 4 - 1
T_FIX = 73
I_TARGET = 7.5
LAST_POINT = T - T_FIX + 1

# Готовим трехмерный объект для хранения 1000 контрфактических итераций по 4 переменным
CF = np.zeros((T, N_ITER, N_VARS))

# Готовим объект чтобы потом мочь посмотреть новые шоки ДКП
CF_EPS_MP = np.zeros((N_ITER, T))

for i in range(N_ITER):
    # Распаковываем параметры каждой итерации
    beta_i = beta[:, i]
    
    # Восстанавливаем коэффициенты VAR
    B_p = beta_i[0:10]
    B_y = beta_i[10:20]
    B_i = beta_i[20:30]
    B_b = beta_i[30:40]
    
    # Структурная матрица A0^{-1} для каждой итерации
    A0inv_i_full = np.vstack([A0inv_p[i, :], A0inv_y[i, :], A0inv_i[i, :],  A0inv_b[i, :]])
    
    # Шоки
    shocks_as = shocks_AS[i, :].reshape(-1, 1)
    shocks_ad = shocks_PrivateAD[i, :].reshape(-1, 1)
    shocks_mp = shocks_MP[i, :].reshape(-1, 1)
    shocks_fp = shocks_FP[i, :].reshape(-1, 1)
    
    # Восстанавливаем начальные значения переменных до t_fix
    Y_temp = np.zeros((T, N_VARS))
    
    for t in range(T_FIX):
        x_t = Xmat[t, :].reshape(-1, 1)
        eps_t = np.vstack([
            shocks_as[t],
            shocks_ad[t],
            shocks_mp[t],
            shocks_fp[t]
        ])
        
        Y_temp[t, P] = (B_p.T @ x_t + A0inv_i_full[P, :] @ eps_t).item()
        Y_temp[t, Y] = (B_y.T @ x_t + A0inv_i_full[Y, :] @ eps_t).item()
        Y_temp[t, I] = (B_i.T @ x_t + A0inv_i_full[I, :] @ eps_t).item()
        Y_temp[t, B] = (B_b.T @ x_t + A0inv_i_full[B, :] @ eps_t).item()
    
    # Считаем контрфакт с момента фиксации
    for t in range(T_FIX, T):
        # Собираем лаги (размер (4,))
        l1 = Y_temp[t-1, :]
        l2 = Y_temp[t-2, :]

        # Формируем x_sim как вектор [10 x 1]
        x_sim = np.vstack([l1.reshape(-1, 1), l2.reshape(-1, 1), Xmat[t, 8], Xmat[t, 9]])
        
        # Вычисляем шоки
        eps_as = shocks_as[t]
        eps_ad = shocks_ad[t]
        eps_fp = shocks_fp[t]
        
        # Вычисляем eps_mp
        numerator = I_TARGET - (B_i.T @ x_sim + A0inv_i_full[I, [0, 1, 3]] @ np.vstack([eps_as, eps_ad, eps_fp]))
        eps_mp = (numerator / A0inv_i_full[I, 2]).item()
        
        # Собираем вектор шоков [4 x 1]
        eps_t = np.vstack([eps_as, eps_ad, eps_mp, eps_fp])
        
        # Обновляем Y_temp
        Y_temp[t, P] = (B_p.T @ x_sim + A0inv_i_full[P, :] @ eps_t).item()
        Y_temp[t, Y] = (B_y.T @ x_sim + A0inv_i_full[Y, :] @ eps_t).item()
        Y_temp[t, I] = I_TARGET  # фиксируем ставку
        Y_temp[t, B] = (B_b.T @ x_sim + A0inv_i_full[B, :] @ eps_t).item()
        
        # Обновляем шок MP
        shocks_mp[t] = eps_mp.item() if isinstance(eps_mp, np.ndarray) else eps_mp
    
    # Сохраняем результат симуляции
    CF[:, i, :] = Y_temp
    CF_EPS_MP[i, :] = shocks_mp.ravel()

# Анализируем
cf_i = CF[:, :, I]
cf_p = CF[:, :, P]
cf_y = CF[:, :, Y]
cf_b = CF[:, :, B]

p_median = np.nanmedian(cf_p, axis=1)
p_low = np.nanpercentile(cf_p, 16, axis=1)
p_high = np.nanpercentile(cf_p, 84, axis=1)

quarters = [START_DATE + timedelta(days=91.25 * i) for i in range(T)]

# Создаем фигуру
fig = go.Figure()

# Добавляем линии на график
fig.add_trace(go.Scatter(
    x=quarters,
    y=p_median,
    mode='lines',
    name='Контрфакт (КС = 7,5%)',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=quarters[-LAST_POINT:],
    y=p_low[-LAST_POINT:],
    mode='lines',
    name='68% ДИ: нижняя',
    line=dict(color='blue', width=1, dash='dash')
))

fig.add_trace(go.Scatter(
    x=quarters[-LAST_POINT:],
    y=p_high[-LAST_POINT:],
    mode='lines',
    name='68% ДИ: верхняя',
    line=dict(color='blue', width=1, dash='dash')
))

fig.add_trace(go.Scatter(
    x=quarters,
    y=actual_p,
    mode='lines',
    name='Фактическая',
    line=dict(color='black', width=1.5)
))

# Настраиваем оформление
fig.update_layout(
    title='Инфляция, QoQ: фактическая vs контрфактическая (КС = 7,5% с 3 квартала 2023 г.)',
    xaxis_title='Квартал',
    yaxis_title='Инфляция (%)',
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    hovermode="x unified",
    margin=dict(l=50, r=50, b=50, t=50),
    xaxis_range=[quarters[0], quarters[-1]],
    yaxis_range=[actual_p.min() - 1, actual_p.max() + 1],
    plot_bgcolor='white',
    showlegend=True
)

# Добавляем сетку
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

# Показать график
fig.show()

fig.write_html(SAVE_PATH + "Инфляция, QoQ. фактическая vs контрфактическая (КС = 7,5% с 3 квартала 2023 г.).html")